# 08 - Device Utilities

This notebook demonstrates the utility functions in `nidaqwrapper.utils` for device discovery and task management.

These functions help you:
- Discover connected NI-DAQmx hardware
- List tasks saved in NI MAX
- Load pre-configured tasks
- Access the UNITS dictionary for physical units

---

In [ ]:
# Mock setup: Try to import nidaqmx, fall back to mock if unavailable
import sys
from unittest.mock import MagicMock, PropertyMock

HW_AVAILABLE = True

try:
    import nidaqmx
    print("Running with real nidaqmx hardware support")
except ImportError:
    print("nidaqmx not available - using mock mode")
    HW_AVAILABLE = False
    
    # Create mock nidaqmx module
    nidaqmx = MagicMock()
    
    # Mock devices
    mock_dev1 = MagicMock()
    mock_dev1.name = "Dev1"
    mock_dev1.product_type = "PCIe-6320"
    
    mock_cdaq = MagicMock()
    mock_cdaq.name = "cDAQ1Mod1"
    mock_cdaq.product_type = "NI 9234"
    
    # Mock tasks
    mock_task1 = MagicMock()
    mock_task1._name = "MyInputTask"
    mock_task1.load = MagicMock(return_value=MagicMock())
    
    mock_task2 = MagicMock()
    mock_task2._name = "MyOutputTask"
    mock_task2.load = MagicMock(return_value=MagicMock())
    
    # Mock system
    mock_system = MagicMock()
    mock_system.devices = [mock_dev1, mock_cdaq]
    mock_system.tasks = [mock_task1, mock_task2]
    mock_system.tasks.task_names = ["MyInputTask", "MyOutputTask"]
    
    nidaqmx.system.System.local = MagicMock(return_value=mock_system)
    
    # Mock constants for UNITS
    nidaqmx.constants = MagicMock()
    nidaqmx.constants.AccelSensitivityUnits.MILLIVOLTS_PER_G = 12506
    nidaqmx.constants.AccelUnits.G = 10356
    nidaqmx.constants.AccelUnits.METERS_PER_SECOND_SQUARED = 10357
    nidaqmx.constants.ForceIEPESensorSensitivityUnits.MILLIVOLTS_PER_NEWTON = 15891
    nidaqmx.constants.ForceUnits.NEWTONS = 15875
    nidaqmx.constants.VoltageUnits.VOLTS = 10348
    
    sys.modules['nidaqmx'] = nidaqmx
    sys.modules['nidaqmx.constants'] = nidaqmx.constants
    sys.modules['nidaqmx.system'] = nidaqmx.system

# Now import nidaqwrapper (works with real or mock nidaqmx)
from nidaqwrapper.utils import (
    list_devices,
    get_connected_devices,
    list_tasks,
    get_task_by_name,
    UNITS
)

## Device Discovery

### list_devices()

Returns a list of all connected NI-DAQmx devices with their product types.

**Signature:** `list_devices() -> list[dict[str, str]]`

Each dict contains:
- `"name"` — device identifier (e.g., `"Dev1"`, `"cDAQ1Mod1"`)
- `"product_type"` — hardware model (e.g., `"PCIe-6320"`, `"NI 9234"`)

Returns an empty list if no devices are connected.

In [ ]:
# List all connected devices
devices = list_devices()
print(f"Found {len(devices)} device(s):")
for device in devices:
    print(f"  {device['name']}: {device['product_type']}")

### get_connected_devices()

Returns just the device names as a set for fast membership checks.

**Signature:** `get_connected_devices() -> set[str]`

Useful when you need to verify if a specific device is connected:
```python
if "Dev1" in get_connected_devices():
    # proceed with Dev1
```

In [ ]:
# Get device names only
device_names = get_connected_devices()
print(f"Connected devices: {device_names}")

# Fast membership check
if "Dev1" in device_names:
    print("Dev1 is available")
else:
    print("Dev1 not found")

## Task Management

### list_tasks()

Returns the names of all tasks saved in NI Measurement & Automation Explorer (NI MAX).

**Signature:** `list_tasks() -> list[str]`

Returns an empty list if no tasks are saved in NI MAX.

In [ ]:
# List all saved tasks
tasks = list_tasks()
print(f"Found {len(tasks)} saved task(s):")
for task in tasks:
    print(f"  {task}")

### get_task_by_name()

Loads a pre-configured task from NI MAX by name.

**Signature:** `get_task_by_name(name: str) -> nidaqmx.task.Task | None`

**Returns:**
- `nidaqmx.Task` object ready to use
- `None` if the task is already loaded by another process

**Raises:**
- `KeyError` — task name not found in NI MAX
- `ConnectionError` — device disconnected or unavailable
- `RuntimeError` — nidaqmx not installed

In [ ]:
# Try to load a saved task
task_name = "MyInputTask"  # Change to an actual task name from list_tasks()

try:
    task = get_task_by_name(task_name)
    if task is None:
        print(f"Task '{task_name}' is already loaded by another process")
    else:
        print(f"Successfully loaded task: {task_name}")
        print(f"Task object: {task}")
        # Don't forget to close when done:
        # task.close()
except KeyError as e:
    print(f"Task not found: {e}")
except ConnectionError as e:
    print(f"Device unavailable: {e}")

## UNITS Dictionary

The `UNITS` dictionary maps physical unit strings to nidaqmx constant enum values.

**Available units:**
- `"mV/g"` — millivolts per g (accelerometer sensitivity)
- `"mV/m/s**2"` — millivolts per m/s² (currently maps to mV/g due to nidaqmx 1.4.1 limitations)
- `"g"` — acceleration in g
- `"m/s**2"` — acceleration in m/s²
- `"mV/N"` — millivolts per newton (force sensor sensitivity)
- `"N"` — force in newtons
- `"V"` — voltage in volts

**Empty dict if nidaqmx is not installed.**

In [ ]:
# Display all available units
print("Available physical units:")
if UNITS:
    for unit_str, unit_constant in UNITS.items():
        print(f"  {unit_str:15s} -> {unit_constant}")
else:
    print("  (empty - nidaqmx not installed)")

### Using UNITS with AITask

The UNITS dictionary is used when adding channels to specify sensor sensitivities and measurement units:

```python
from nidaqwrapper import AITask

task = AITask()
task.add_channel(
    name="Dev1/ai0",
    channel_type="accel",
    units="g",              # From UNITS dict
    sensitivity=100.0,
    sensitivity_units="mV/g"  # From UNITS dict
)
```

The wrapper converts these strings to the correct nidaqmx constants internally.

## Practical Workflow

Typical usage pattern for device discovery and task loading:

1. **Discover hardware** — see what's connected
2. **List saved tasks** — see what configurations exist
3. **Load and use** — load a task or create new channels

In [ ]:
# Complete discovery workflow
print("=== Hardware Discovery ===")
devices = list_devices()
if devices:
    print(f"\nFound {len(devices)} device(s):")
    for dev in devices:
        print(f"  - {dev['name']} ({dev['product_type']})")
else:
    print("No devices connected")

print("\n=== Saved Tasks ===")
tasks = list_tasks()
if tasks:
    print(f"\nFound {len(tasks)} saved task(s):")
    for task in tasks:
        print(f"  - {task}")
else:
    print("No tasks saved in NI MAX")

print("\n=== Available Units ===")
if UNITS:
    print(f"\n{len(UNITS)} unit types available:")
    print(f"  {list(UNITS.keys())}")
else:
    print("No units available (nidaqmx not installed)")

## Error Handling

All utility functions raise `RuntimeError` if nidaqmx is not installed:

```python
RuntimeError: NI-DAQmx drivers are required for this operation. 
              Install the package with: pip install nidaqmx
```

Additional exceptions from `get_task_by_name()`:
- `KeyError` — task name doesn't exist in NI MAX (message includes available task names)
- `ConnectionError` — device is disconnected or in use by another application

In [ ]:
# Example: handling task loading errors
try:
    task = get_task_by_name("NonExistentTask")
except KeyError as e:
    print(f"Task not found error (expected):\n  {e}")
except ConnectionError as e:
    print(f"Connection error:\n  {e}")
except RuntimeError as e:
    print(f"Runtime error (nidaqmx not installed):\n  {e}")

## Summary

The utility functions provide essential hardware discovery and task management:

| Function | Returns | Purpose |
|----------|---------|----------|
| `list_devices()` | `list[dict[str, str]]` | Full device info (name + product type) |
| `get_connected_devices()` | `set[str]` | Device names only (fast membership check) |
| `list_tasks()` | `list[str]` | Task names saved in NI MAX |
| `get_task_by_name(name)` | `Task \| None` | Load pre-configured task |
| `UNITS` | `dict[str, Any]` | Physical unit string → nidaqmx constant |

**All functions require nidaqmx to be installed and will raise `RuntimeError` otherwise.**